In [1]:
from src.common.vector import *
from src.common.reference_frame import *

r = Vector(-5368,-1784,3691)
(ra,dec) = radec_from_r(r)

print(f'RA = {np.rad2deg(ra)}\nDEC = {np.rad2deg(dec)}')

RA = 198.38370037548617
DEC = 33.12454287112769


In [2]:
from src.common.vector import *
from src.common.reference_frame import *
from src.common.time import *
from datetime import datetime

date_1 = datetime(2004,5,12,14,45,30)
jd_1 = julian_day(date_1)
date_2 = datetime(1957,10,4,19,26,24)
jd_2 = julian_day(date_2)
print(f'JD_1 = {jd_1} dias\nJD_2 = {jd_2} dias')
print(f'Diferença = {jd_1 - jd_2} dias')


JD_1 = 2453138.1149305557 dias
JD_2 = 2436116.31 dias
Diferença = 17021.804930555634 dias


In [3]:
from src.common.vector import *
from src.common.time import *
from datetime import datetime

theta = sidereal_time(date=datetime(2004,3,3,4,30,0),longitude=139.8)
theta

8.593542535242534

In [4]:
from src.common.vector import *
from src.common.reference_frame import *
import math

r = Vector(x=-5368,y=-1784,z=3691)
r_o = get_topocentric_origin(60,20,0,theta_g=126.7)
print(f'r = {round(r_o.x,0)}I + ({round(r_o.y,1)})J + ({round(r_o.z,0)})K')
rho = r-r_o
ra,dec = radec_from_r(rho)
print(f'RA = {round(np.rad2deg(ra),1)}°\nDEC = {round(np.rad2deg(dec),2)}°')

r = -5955.0I + (-699.5)J + (2168.0)K
RA = 298.4°
DEC = 51.01°


In [5]:
print(r-r_o)

x = 586.7609386828062
y = -1084.4763599394746
z = 1523.3505491892229
Magnitude = 1959.8403685368376
Theta_x = 72.57889850094928
Theta_y = 123.59709601047628
Theta_z = 38.98752421498512
